In [1]:
import pandas as pd
from dotenv import load_dotenv
import subprocess
import os
from extractor import process_image_with_together
import re
from text_mine import extract_data_flexible


In [16]:
import subprocess
file_path = "bill.jpeg"
api_key = "21c8753437eb9ce5822db4ee270abea087cc4ab2e31f59401f8ee91d1973aa62"
result = subprocess.run(
    ['node','ocrScript.js',file_path,api_key],
    capture_output=True,
    text= True
)

print(result.stdout)

In [18]:
!npm install together-ai


added 53 packages in 27s

3 packages are looking for funding
  run `npm fund` for details


npm warn deprecated q@1.5.1: You or someone you depend on is using Q, the JavaScript Promise library that gave JavaScript developers strong feelings about promises. They can almost certainly migrate to the native JavaScript promise now. Thank you literally everyone for joining me in this bet against the odds. Be excellent to each other.
npm warn deprecated
npm warn deprecated (For a CapTP with native promises, see @endo/eventual-send and @endo/captp)
npm notice
npm notice New patch version of npm available! 10.9.0 -> 10.9.1
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.9.1
npm notice To update run: npm install -g npm@10.9.1
npm notice


In [23]:
from together import Together
import base64

client = Together(api_key="21c8753437eb9ce5822db4ee270abea087cc4ab2e31f59401f8ee91d1973aa62")

getDescriptionPrompt = """extract date,Rate,product,amount(rs),quantity. date is alway in dd-mm-yyyy format. provide key-value pair"""

#imagePath= "U:/Bill_image/1_Screenshot_20241001-100107.png"
imagePath= "i7.jpg"

def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

base64_image = encode_image(imagePath)

stream = client.chat.completions.create(
    model="meta-llama/Llama-Vision-Free",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": getDescriptionPrompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        }
    ],
    stream=True,
)
result =""

for chunk in stream:
    if chunk.choices and len(chunk.choices) > 0:
        delta = getattr(chunk.choices[0], "delta", None)
        content = getattr(delta, "content", None)
        if content:
            result += content  
            print(content, end="", flush=True)
    else:
        print("Invalid chunk structure")




Key-Value Pairs:

* Date: 18-11-2024
* Rate: 103
* Product: Petrol
* Amount (Rs.): 3090
* Quantity: 30

These key-value pairs are extracted from the Cash/Credit Memo document provided.Invalid chunk structure


In [37]:
import json
from together import Together
import base64

client = Together(api_key="21c8753437eb9ce5822db4ee270abea087cc4ab2e31f59401f8ee91d1973aa62")

getDescriptionPrompt = """detect and give all important info.for  date use dd-mm-yyyy format"""

imagePath = "test.jpg"  # Image path

# Function to encode the image into base64 format
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

base64_image = encode_image(imagePath)

# Making the API request
try:
    stream = client.chat.completions.create(
        model="meta-llama/Llama-Vision-Free",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": getDescriptionPrompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    },
                ],
            }
        ],
        stream=True,
    )

    result = ""  # To accumulate result content
    result_data = {}  # Dictionary to store the final result

    # Process the stream to extract content
    for chunk in stream:
        if chunk.choices and len(chunk.choices) > 0:
            delta = getattr(chunk.choices[0], "delta", None)
            content = getattr(delta, "content", None)
            if content:
                result += content
                print(content, end="", flush=True)
                # Assuming the content contains structured data, you can store it in result_data
                # You can parse it as key-value pairs if needed, or just store the entire result as text
                result_data["extracted_text"] = result  # Save the content as part of the result

        else:
            print("Invalid chunk structure")

    # Save the result as a JSON file
    with open('ocr_result.json', 'w') as json_file:
        json.dump(result_data, json_file, indent=4)

    print("\nResult saved to 'ocr_result.json'")

except Exception as e:
    print(f"Error occurred: {e}")


**Receipt Details**

* **Date:** 03-08-2024
* **Location:** M/s. K.G. Bhat Service Station, Pumpwell, Mangaluru - 575 002
* **Dealer Information:**
	+ Name: Indian Oil Corporation Ltd.
	+ Address: Pumpwell, Mangaluru - 575 002
* **Payment Information:**
	+ Amount Paid: 2000.00
	+ Payment Method: Cash/Credit Memo
* **Vehicle Information:**
	+ Vehicle No.: Not Provided
	+ Particulars: M.S. H.S.D. XP95
	+ Rate: 88.15
* **Quantity Information:**
	+ Qty.: 22.62 Ltr
* **Memo Number:** No: 106413Invalid chunk structure

Result saved to 'ocr_result.json'


In [31]:
data = extract_data_flexible(result)



In [28]:
data

{'Bill_Date': '03-08-2024',
 'Rate': '88.15',
 'Quantity': '22.68',
 'Amount': '2000.00',
 'Product': 'LUBE'}

In [37]:
data1 = pd.DataFrame([data])

In [38]:
data1

,Bill_Date,Rate,Quantity,Amount,Product
0,03-08-2024,88.15,22.62 liters,2000.00,LUBE


In [26]:
print(result)
if "Date" in result:
    # Extract the date in the format dd/mm/yyyy using regex
    date_match = re.findall(r"\b(\d{2}/\d{2}/\d{4})\b", result)
    if date_match:
        extracted_date = date_match[0]  # Get the first match (if multiple, handle accordingly)
        print("Extracted Date:", extracted_date)

The invoice details are as follows:

**Date:** 10-10-2024
**Time:** 22:20:02
**Location:** Thane, Mainpada
**Vehicle Information:**
* **Make:** Laxmi Automobiles
* **Model:** Thane, 400607
* **Chassis Number:** 4101011378
* **Engine Number:** Not provided
* **Vehicle Type:** Petrol (2 Wheeler)

**Transaction Details:**
* **Amount:** 2000.00
* **Currency:** Indian Rupee (INR)
* **Tax Type:** Local Amount
* **Rate/Ltr.:** 103.57
* **Volume (Ltr.):** 19.31
* **TxEnd:** 22:20:02
* **TxSt:** 22:20:02
* **Attendant:** Not provided
* **PayMode:** Cash
* **Product:** Petrol
* **NozzleNo:** Not provided
* **BayNo:** Not provided
* **Time:** 22:22:04


In [ ]:
result

AttributeError: 'builtin_function_or_method' object has no attribute 'splitlines'

In [33]:
result

<function str.splitlines(keepends=False)>

In [ ]:
columns = ["Bill Number", "Bill Date", "Amount", "Rate", "Quantity", "Time"]

# Process the result into key-value pairs
data = {}
for line in result.strip().split("\n"):
    if ":**" in line:
        key, value = line.split(":**")
        key = key.strip("** ").strip()
        value = value.strip()
        data[key] = value

# Ensure data matches the predefined columns (use empty string if a column is missing)
row = [data.get(col, "") for col in columns]

# Create a DataFrame with predefined columns
df = pd.DataFrame([row], columns=columns)

# Display the DataFrame
df

,Bill Number,Bill Date,Amount,Rate,Quantity,Time
0,106413,,2000.00,88.15,22.62 Ltrs,Not provided


In [45]:
import re
import pandas as pd



# Define the expected keys and associated patterns
patterns = {
    "Bill Number": r"Bill Number[:\s*]+(\d+)",
    "Bill Date": r"Date[:\s*]+([\d/]+)",
    "Amount": r"Amount[:\s*]+([\d.]+)",
    "Rate": r"Rate[:\s*]+([\d.]+)",
    "Quantity": r"Quantity[:\s*]+([\d.]+)",
    "Time": r"Time[:\s*]+(.+)"
}

# Preprocess the text to remove stars and extra symbols
normalized_result = re.sub(r"[*]+", "", result)  # Remove all asterisks
normalized_result = normalized_result.strip()    # Remove leading/trailing whitespace

# Initialize a dictionary to store extracted data
data = {}

# Iterate over the patterns and extract values using regex
for key, pattern in patterns.items():
    match = re.search(pattern, normalized_result)
    if match:
        data[key] = match.group(1).strip()  # Extract and clean the matched value
    else:
        data[key] = ""  # Default to empty if no match found

# Define the columns and create a DataFrame
columns = ["Bill Number", "Bill Date", "Amount", "Rate", "Quantity", "Time"]
row = [data.get(col, "") for col in columns]
df = pd.DataFrame([row], columns=columns)

# Output the DataFrame
print(df)


   Bill Number Bill Date Amount Rate Quantity      Time
0  27760065781        12                       10:19:38


In [122]:
# Input: text extracted from the image in key:value format



# Dictionary to store the extracted values
data_dict = {
    "date": None,
    "time": None,
    "bill_no": None,
    "amount": None,
    "volume": None,
    "rate": None
}

# Process the text line by line
for line in result.splitlines():
    line = line.strip()  # Remove leading/trailing whitespace  # Debug print to trace the line content

    if line.startswith("* **Date**"):
        data_dict["date"] = line.split(":")[1].strip()
        print(f"Debug: Extracted date: {data_dict['date']}")
    elif line.startswith("* **Time**"):
        data_dict["time"] = line.split(": ", 3)[1].strip()
        print(f"Debug: Extracted time: {data_dict['time']}")
    elif line.startswith("* **BILL NUM**|* **Bay No**"):
        data_dict["bill_no"] = line.split(": ")[1].strip()
        print(f"Debug: Extracted bill number: {data_dict['bill_no']}")
    elif line.startswith("* **BASE AMT.**"):
        data_dict["amount"] = line.split(": ", 1)[1].strip()
        print(f"Debug: Extracted amount: {data_dict['amount']}")
    # Additional fields can be added as needed

# Output the results
print("\nFinal Extracted Data:", data_dict)


Debug: Extracted date: 12-10-2024
Debug: Extracted time: 10:19:38

Final Extracted Data: {'date': '12-10-2024', 'time': '10:19:38', 'bill_no': None, 'amount': None, 'volume': None, 'rate': None}


In [27]:
import re
import pandas as pd

# Example result string
result = """
Date: 12-10-2024
Time: 10:19:38
Product: Petrol
Cash: 4101217576
Txn Id: 4101217576
Attendant: Not specified
TxSt: 12-10-24 10:18:12
TxEnd: 12-10-24 10:18:41
Rate/Ltr.: 103.57
Volume (Ltr.): 3.86
Amount (Rs.): 399.78
VechNo: Not specified
MobileNo: Not specified
"""

# Define patterns for the fields
patterns = {
    "Bill Date": r"Date:\s+([\d-]+)",
    "Time": r"Time:\s+([\d:]+)",
    "Rate": r"Rate/Ltr\.\:\s+([\d.]+)",
    "Quantity": r"Volume \(Ltr\.\):\s+([\d.]+)",
    "Amount": r"Amount \(Rs\.\):\s+([\d.]+)",
    "Bill Number": r"Txn Id:\s+(\d+)"  # Assuming Txn Id is the bill number
}

# Extract the data using regex
data = {}
for key, pattern in patterns.items():
    match = re.search(pattern, result)
    if match:
        data[key] = match.group(1).strip()
    else:
        data[key] = ""  # Default to empty if no match found

# Define the columns and create a DataFrame
columns = ["Bill Date", "Time", "Rate", "Quantity", "Amount", "Bill Number"]
row = [data.get(col, "") for col in columns]
df = pd.DataFrame([row], columns=columns)

# Output the DataFrame
print(df)


    Bill Date      Time    Rate Quantity  Amount Bill Number
0  12-10-2024  10:19:38  103.57     3.86  399.78  4101217576


In [46]:
Data = {"Date":"",
        "Time":"",
        "Bill_No":"",
        "Rate/Ltr":"",
        "Qantity":"",
        "Amount":""}

In [61]:

key_prompts = {
    "Date": "extract date.",
    "Rate/Ltr": "extract Rate/Ltr.",
    "Qantity": "extract Quantity. The quantity and volume are the same.",
    "Amount": "extract Amount in rs."
}

# Loop through the mapping and process the keys dynamically
for key, prompt in key_prompts.items():
    if key in Data:
        extracted_value = process_image_with_together(client, base64_image, prompt)
        Data[key] = extracted_value  
        print(f"{key}: {extracted_value}")


Invalid chunk structure
Date: The date of the receipt is 12-10-2024.
Invalid chunk structure
Rate/Ltr: The Rate/Ltr. is 103.57.
Invalid chunk structure
Qantity: The quantity and volume are both 3.86. 

This suggests that the product being sold is liquid, such as gasoline or diesel, and the amount purchased is 3.86 liters.
Invalid chunk structure
Amount: The amount in rupees is 399.78.


In [62]:
Data

{'Date': 'The date of the receipt is 12-10-2024.',
 'Time': '',
 'Bill_No': '',
 'Rate/Ltr': 'The Rate/Ltr. is 103.57.',
 'Qantity': 'The quantity and volume are both 3.86. \n\nThis suggests that the product being sold is liquid, such as gasoline or diesel, and the amount purchased is 3.86 liters.',
 'Amount': 'The amount in rupees is 399.78.'}